PART OF FIRE -HASOC Competition -One of Four task (rest of the codes are seprate)



## Importing libraries


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pandas as pd

## For quick changes

In [ ]:

location1='/content/drive/MyDrive/hate rate/task 4/model saving/Assamese bert/lr=0.1/model_weights.pth'
location2='/content/drive/MyDrive/hate rate/task 4/model saving/Assamese bert/lr=0.1/model_architecture.pth'




## model

In [ ]:
# df=pd.read_csv('/content/drive/MyDrive/hate rate/task 4/dataset/train_AH_preprocessed.csv')
df=pd.read_csv('/content/drive/MyDrive/hate rate/task 4/dataset/train_BO_AH_HASOC2023.csv')

In [ ]:

import re

def clean_assamese_text(text):
    # Remove emojis and non-Assamese characters
    clean_text = re.sub(r'[^\w\s\u0980-\u09FF]+', ' ', text)

    # Replace multiple spaces with a single space
    clean_text = re.sub(r'\s+', ' ', clean_text)

    return clean_text.strip()


preprocessed_text=pd.Series()
for indx,row in df.iterrows():
  p_text=clean_assamese_text(row['text'])
  preprocessed_text.at[indx]=p_text
df['pre_text']=preprocessed_text

<ipython-input-57-28eba094286f>:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  preprocessed_text=pd.Series()


In [ ]:
df.columns

Index(['S. No.', 'text', 'task_1', 'pre_text'], dtype='object')

In [ ]:
#convert label to int
output=pd.Series()
for indx,row in df.iterrows():
  current=row['task_1']
  if(current=="NOT"):
    output.at[indx]=0
  else:
    output.at[indx]=1
df['output']=output


#finding longest sentence
maxx=0

for s in df['text']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
  maxx=max(s_len,maxx)



# Pad sequences to a fixed length (adjust MAX_LEN if needed)
MAX_LEN = maxx
print(maxx)

<ipython-input-59-f250d2b08d33>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  output=pd.Series()


62


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



# Splitting the dataframe into train and test sets
# _, datadf = train_test_split(df, test_size=0.2, random_state=42)
train_df,val_df=train_test_split(df,test_size=0.1,random_state=57)



# Reset the index of the DataFrame
train_df = train_df.reset_index(drop=True)
# Reset the index of the DataFrame
val_df = val_df.reset_index(drop=True)



In [ ]:
model_name='alayaran/bodo-bert-mlm-base-article'

from transformers import BertModel,AutoTokenizer
# Download pytorch model

tokenizer = AutoTokenizer.from_pretrained(model_name, padding='max_length', max_length=128,return_tensors='pt')

In [ ]:
df.columns

Index(['S. No.', 'text', 'task_1', 'pre_text', 'output'], dtype='object')

In [ ]:
unqiue_word=set()
for sentence in df['text']:
  words=sentence.split()
  for word in words:
    unqiue_word.add(word)
print(len(unqiue_word))
vocab=tokenizer.get_vocab()
print(len(vocab))
yes_count=0
no_count=0
for word in unqiue_word:
  if (word in vocab):
    yes_count+=1
  else:
    no_count+=1

4903
50000


In [ ]:
print(yes_count)
print(no_count)

1222
3681


In [ ]:
bert_model=BertModel.from_pretrained(model_name)

for param in bert_model.encoder.layer.parameters():
            param.requires_grad = False

# for param in bert_model.encoder.layer[-4:].parameters():
#             param.requires_grad = True


for name, param in bert_model.named_parameters():
            print(f'Layer: {name} - Trainable: {param.requires_grad}')

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.7.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.1.attention.self.value.bias', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.5.attention.self.query.bias', 'encoder.layer.7.intermediate.dense.weight', 'encoder.layer.2.attention.output.dense.bias', 'encoder.layer.3.attention.output.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.8.output.dense.weight', 'encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.4.attention.self.key.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.6.attention.self.ke

Layer: embeddings.word_embeddings.weight - Trainable: True
Layer: embeddings.position_embeddings.weight - Trainable: True
Layer: embeddings.token_type_embeddings.weight - Trainable: True
Layer: embeddings.LayerNorm.weight - Trainable: True
Layer: embeddings.LayerNorm.bias - Trainable: True
Layer: encoder.layer.0.attention.self.query.weight - Trainable: False
Layer: encoder.layer.0.attention.self.query.bias - Trainable: False
Layer: encoder.layer.0.attention.self.key.weight - Trainable: False
Layer: encoder.layer.0.attention.self.key.bias - Trainable: False
Layer: encoder.layer.0.attention.self.value.weight - Trainable: False
Layer: encoder.layer.0.attention.self.value.bias - Trainable: False
Layer: encoder.layer.0.attention.output.dense.weight - Trainable: False
Layer: encoder.layer.0.attention.output.dense.bias - Trainable: False
Layer: encoder.layer.0.attention.output.LayerNorm.weight - Trainable: False
Layer: encoder.layer.0.attention.output.LayerNorm.bias - Trainable: False
Layer: 

In [ ]:
def get_embeddings(row):
    text=row['pre_text']



    text_token=tokenizer(text,truncation=True,return_tensors='pt',padding='max_length',max_length=128)



    text_input={'input_ids':text_token['input_ids'],'attention_mask':text_token['attention_mask']}


    return text_input

next time add a attention layer

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class our_model(nn.Module):
     def __init__(self,bert_modell):
         super(our_model, self).__init__()

         # Define BERT models
         self.bert_model = bert_modell


         # Define LSTM
         self.lstm1 = nn.LSTM(input_size=768, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)

         self.lstm2 = nn.LSTM(input_size=2*128, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
         self.lstm3 = nn.LSTM(input_size=2*128, hidden_size=64, num_layers=1, bidirectional=False, batch_first=True)
         self.lstm4 = nn.LSTM(input_size=64, hidden_size=64, num_layers=1, bidirectional=False, batch_first=True)




         self.dense_layers = nn.Sequential(
             nn.Linear(64, 32),
             nn.ReLU(),
             nn.Dropout(0.3),
             nn.Linear(32, 1),
         )
         #self.dropout=nn.Dropout(0.4)
         self.sigmoid = nn.Sigmoid()

     def forward(self, text):


        text_input_ids = text['input_ids'].squeeze(1).to(device)
        text_attention_mask = text['attention_mask'].squeeze(1).to(device)#


         # BERT outputs
        bert_output = self.bert_model(input_ids=text_input_ids, attention_mask=text_attention_mask).last_hidden_state


#         # LSTM
        text_lstm_output,(_,_) = self.lstm1(bert_output)
        text_lstm_output,(_,_) = self.lstm2(text_lstm_output)

        text_lstm_output,(_,_) = self.lstm3(text_lstm_output)


        #layer 3
        _,(text_lstm_output,_)=self.lstm4(text_lstm_output)



         # Dense layers
        dense_output = self.dense_layers(text_lstm_output)

        output = self.sigmoid(dense_output)
        output=output.squeeze(0)
        return output

In [ ]:
# Check if a GPU device is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
batch_size=16

In [ ]:

# Create a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        label = row['output']
        text_input = get_embeddings(row)
        return {'text_input':text_input, 'label': label}

# Create an instance of the custom dataset
dataset_train = CustomDataset(train_df)
# Create a DataLoader to handle batching and shuffling of the data
train_data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = CustomDataset(val_df)

val_data_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True)


In [ ]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch

# Create a new model instance with the extracted BERT layer weights
model = our_model(bert_model)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


our_model(
  (bert_model): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
     

the below code is working good 😊



In [ ]:
model_stats=pd.DataFrame(columns=['epoch','val accuracy','train accuracy','train_loss','val_loss','learning rate'])

## training the model

In [ ]:
train_batch_number=len(train_data_loader)
val_batch_number=len(val_data_loader)
print(train_batch_number)
print(val_batch_number)

227
26


In [ ]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
threshold=0.5
n_epochs=10
loss_fn = nn.BCELoss()
# Train the model
optimizer = optim.AdamW(model.parameters(), lr=0.0000001, weight_decay=0.001)

model.train()
total_data=len(train_df)
prev_total_loss = None  # Variable to store the previous total_loss
best_accuracy=0
a=0
for epoch in range(n_epochs):
    total_loss = 0
    total_samples_train = 0
    batch_c = 0
    correct_predictions_train = 0
    optimizer.zero_grad()

    for batch in train_data_loader:
        batch_c += 1
        text_input = batch['text_input']

        y_true = batch['label'].unsqueeze(1).float().to(device)
        y_pred = model(text_input)


        loss = loss_fn(y_pred, y_true)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        predicted_labels_train = (y_pred >= threshold).float()

        total_samples_train += len(y_true)
        for j in range(len(predicted_labels_train)):
            if y_true[j] == predicted_labels_train[j]:
                correct_predictions_train += 1






    correct_predictions_val = 0
    total_samples_val = 0
    total_val_loss=0
    for val_batch in val_data_loader:
        text_input = val_batch['text_input']

        y_true = val_batch['label'].unsqueeze(1).float().to(device)
        y_pred = model(text_input)

        predicted_labels = (y_pred >= threshold).float()
        total_samples_val += len(y_true)
        val_loss=loss_fn(y_pred,y_true)
        total_val_loss+=val_loss.item()
        for j in range(len(predicted_labels)):
            if y_true[j] == predicted_labels[j]:
                correct_predictions_val += 1
    print("**********************")
    total_val_loss/=val_batch_number



    val_accuracy=(correct_predictions_val / total_samples_val) * 100

    train_accuracy=(correct_predictions_train / total_samples_train) * 100

    learning_rate=optimizer.param_groups[0]['lr']

    total_loss/=train_batch_number
    print('Epoch {}.   train accuracy {}.      train loss: {}.     val accuracy : {}.    val loss :  {}.     lr : {}.  '.format(epoch,train_accuracy,total_loss,val_accuracy,total_val_loss,learning_rate))
    new_row={'epoch':epoch,'val accuracy':val_accuracy,'train accuracy':train_accuracy,'train_loss':total_loss,'val_loss':total_val_loss,'learning rate':learning_rate}
    model_stats=model_stats.append(new_row,ignore_index=True)
    # if ( val_accuracy > best_accuracy) :
    #     best_accuracy = val_accuracy
    #     torch.save(model.state_dict(), location1)
    #     print(f"Saved the best model with validation accuracy: {best_accuracy:.2f}")
    #     torch.save(model, location2)






KeyboardInterrupt: ignored